In [3]:
import pandas as pd
import numpy as np

In [4]:
url = "https://opendata.rhein-kreis-neuss.de/api/v2/catalog/datasets/rhein-kreis-neuss-ladesaulen-in-deutschland/exports/csv"
df = pd.read_csv(url, delimiter=";")


In [5]:
list(df)

['betreiber',
 'art_der_ladeeinrichung',
 'anzahl_ladepunkte',
 'anschlussleistung',
 'steckertypen1',
 'steckertypen2',
 'steckertypen3',
 'steckertypen4',
 'p1_kw',
 'p2_kw',
 'p3_kw',
 'p4_kw',
 'kreis_kreisfreie_stadt',
 'ort',
 'postleitzahl',
 'strasse',
 'hausnummer',
 'adresszusatz',
 'inbetriebnahmedatum',
 'koordinaten']

In [6]:
#list(df)

In [7]:
from geopy.geocoders import Nominatim
def get_state_by_coord(coord):
    geolocator = Nominatim(user_agent="geoapi")
    location = geolocator.reverse(coord, exactly_one=True)
    address = location.raw['address']
    state = address.get('state', '')
    return state

In [8]:
selected_columns = ["betreiber", "anzahl_ladepunkte", "anschlussleistung", "koordinaten"]
df = df[selected_columns].rename(columns={"betreiber": "operator", "anzahl_ladepunkte": "number_of_charging_point", "anschlussleistung" : "connected_load", 
                                          "koordinaten" : "coordinates"})

In [15]:
#df["coordinates"] = df["coordinates"].map(get_state_by_coord)

df_new = df.iloc[:10]
df_new["coordinates"] = df_new["coordinates"].map(get_state_by_coord, na_action='ignore')
df_new.rename(columns={"coordinates" : "State"}, inplace=True)
df_new.to_csv("./mobi-data-tmp.csv", index=False)
df_new

/var/folders/jt/lp68flw167df0vb5tj2dgvfc0000gn/T/ipykernel_8253/2955718872.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["coordinates"] = df_new["coordinates"].map(get_state_by_coord, na_action='ignore')
/var/folders/jt/lp68flw167df0vb5tj2dgvfc0000gn/T/ipykernel_8253/2955718872.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.rename(columns={"coordinates" : "State"}, inplace=True)


,operator,number_of_charging_point,connected_load,State
0,GGEW AG,1,4.6,Hessen
1,Pfalzwerke AG,2,75.0,Hessen
2,ENTEGA Energie GmbH,2,44.0,Hessen
3,Entega Plus GmbH,2,22.0,Hessen
4,Entega Plus GmbH,2,22.0,Hessen
5,Vogel Autohäuser GmbH & Co. KG,2,44.0,Hessen
6,Entega Plus GmbH,2,150.0,Hessen
7,Entega Plus GmbH,2,22.0,Hessen
8,ALDI SÜD,2,44.0,Hessen
9,ENTEGA Energie GmbH,2,44.0,Hessen


In [14]:
import os.path as path
file_path = "./mobi-data-tmp.csv"
if not path.isfile(file_path):
    print("Yes it present")
else:
    print("No it is not")

Yes it present


In [16]:
df_new = df_new.drop(columns=["operator"])
df_new.head()

,number_of_charging_point,connected_load,State
0,1,4.6,Hessen
1,2,75.0,Hessen
2,2,44.0,Hessen
3,2,22.0,Hessen
4,2,22.0,Hessen


In [17]:
df_new = df_new.groupby(["State"]).agg({"number_of_charging_point": sum, "connected_load": sum}).reset_index()
df_new

,State,number_of_charging_point,connected_load
0,Hessen,19,471.6


In [18]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///../tmp.sqlite', echo=False)
df_new.to_sql("mobi", con=engine, if_exists="replace", index=False)

1

In [ ]:
db = pd.read_sql_table("mobi", "sqlite:///tmp.sqlite")
len(db.index)

2

In [ ]:
df = pd.read_excel("fz28_2023_04.xlsx;jsessionid=3FA0B971D18E156CCB2B0AC11B4C40BF.xlsx", sheet_name="FZ 28.9", usecols="B:C")
df.rename(columns={"Unnamed: 1": "State", "Unnamed: 2": "EV Sales"}, inplace=True)
df = df.iloc[12:28]
df

,State,EV Sales
12,Baden-Württemberg,29256
13,Bayern,43821
14,Berlin,6367
15,Brandenburg,3349
16,Bremen,1349
17,Hamburg,5907
18,Hessen,25011
19,Mecklenburg-Vorpommern,2150
20,Niedersachsen,19412
21,Nordrhein-Westfalen,39416


In [ ]:
# ["Unnamed: 1", "Unnamed: 2"]
df1 = pd.read_excel("https://www.kba.de/SharedDocs/Downloads/DE/Statistik/Fahrzeuge/FZ28/fz28_2022_10.xlsx?__blob=publicationFile&v=4", sheet_name="FZ 28.9", usecols="B:C")
df1.rename(columns={"Unnamed: 1": "State", "Unnamed: 2": "EV Sales"}, inplace=True)
df1 = pd.DataFrame(df1.iloc[12:28])
df1.reset_index()
df1

,State,EV Sales
12,Baden-Württemberg,29845
13,Bayern,42870
14,Berlin,6050
15,Brandenburg,3692
16,Bremen,1168
17,Hamburg,6598
18,Hessen,22815
19,Mecklenburg-Vorpommern,1930
20,Niedersachsen,20114
21,Nordrhein-Westfalen,44174


In [ ]:
for ind in df1.index:
    index = df.index[df1["State"][ind]==df["State"]].to_list()[0]
    df["EV Sales"][index] += df1["EV Sales"][ind]
    
df

,State,EV Sales
12,Baden-Württemberg,59101
13,Bayern,86691
14,Berlin,12417
15,Brandenburg,7041
16,Bremen,2517
17,Hamburg,12505
18,Hessen,47826
19,Mecklenburg-Vorpommern,4080
20,Niedersachsen,39526
21,Nordrhein-Westfalen,83590


In [ ]:
s = "betreiber;art_der_ladeeinrichung;anzahl_ladepunkte;anschlussleistung;steckertypen1;steckertypen2;steckertypen3;steckertypen4;p1_kw;p2_kw;p3_kw;p4_kw;kreis_kreisfreie_stadt;ort;postleitzahl;strasse;hausnummer;adresszusatz;inbetriebnahmedatum;koordinaten"
s.split(";")

['betreiber',
 'art_der_ladeeinrichung',
 'anzahl_ladepunkte',
 'anschlussleistung',
 'steckertypen1',
 'steckertypen2',
 'steckertypen3',
 'steckertypen4',
 'p1_kw',
 'p2_kw',
 'p3_kw',
 'p4_kw',
 'kreis_kreisfreie_stadt',
 'ort',
 'postleitzahl',
 'strasse',
 'hausnummer',
 'adresszusatz',
 'inbetriebnahmedatum',
 'koordinaten']

In [1]:
import pandas as pd

In [2]:
url = "https://download-data.deutschebahn.com/static/datasets/haltestellen/D_Bahnhof_2020_alle.CSV"
df = pd.read_csv(url, delimiter=';')
df.head()

,EVA_NR,DS100,IFOPT,NAME,Verkehr,Laenge,Breite,Betreiber_Name,Betreiber_Nr,Status
0,8002551,AELB,de:02000:11943,Hamburg Elbbrücken,RV,"10,0245","53,5345",DB Station und Service AG,NaN,neu
1,8001944,TETN,NaN,Eutingen Nord,RV,"8,7531","48,4847",DB Station und Service AG,NaN,neu
2,8003074,MIA,NaN,Ingolstadt Audi,RV,"11,4074564","48,7904959",DB Station und Service AG,NaN,neu
3,8001723,HEBA,NaN,Einbeck Otto-Hahn-Straße,RV,"9,89290953","51,8144784",Ilmebahn GmbH,NaN,neu
4,8004371,KRO,NaN,Nörvenich-Rommelsheim,nur DPN,"6,547586","50,782539",Rurtalbahn GmbH,NaN,neu


In [3]:
df.drop('Status', axis=1, inplace=True)
df.head()

,EVA_NR,DS100,IFOPT,NAME,Verkehr,Laenge,Breite,Betreiber_Name,Betreiber_Nr
0,8002551,AELB,de:02000:11943,Hamburg Elbbrücken,RV,"10,0245","53,5345",DB Station und Service AG,NaN
1,8001944,TETN,NaN,Eutingen Nord,RV,"8,7531","48,4847",DB Station und Service AG,NaN
2,8003074,MIA,NaN,Ingolstadt Audi,RV,"11,4074564","48,7904959",DB Station und Service AG,NaN
3,8001723,HEBA,NaN,Einbeck Otto-Hahn-Straße,RV,"9,89290953","51,8144784",Ilmebahn GmbH,NaN
4,8004371,KRO,NaN,Nörvenich-Rommelsheim,nur DPN,"6,547586","50,782539",Rurtalbahn GmbH,NaN


In [4]:
valid_set = ["FV", "RV", "nur DPN"]
df = df[df['Verkehr'].isin(valid_set)]
df.head()

,EVA_NR,DS100,IFOPT,NAME,Verkehr,Laenge,Breite,Betreiber_Name,Betreiber_Nr
0,8002551,AELB,de:02000:11943,Hamburg Elbbrücken,RV,"10,0245","53,5345",DB Station und Service AG,NaN
1,8001944,TETN,NaN,Eutingen Nord,RV,"8,7531","48,4847",DB Station und Service AG,NaN
2,8003074,MIA,NaN,Ingolstadt Audi,RV,"11,4074564","48,7904959",DB Station und Service AG,NaN
3,8001723,HEBA,NaN,Einbeck Otto-Hahn-Straße,RV,"9,89290953","51,8144784",Ilmebahn GmbH,NaN
4,8004371,KRO,NaN,Nörvenich-Rommelsheim,nur DPN,"6,547586","50,782539",Rurtalbahn GmbH,NaN


In [4]:
len(df.index)

6519

In [5]:
df_tmp = df.iloc[:50]
df_tmp

,EVA_NR,DS100,IFOPT,NAME,Verkehr,Laenge,Breite,Betreiber_Name,Betreiber_Nr
0,8002551,AELB,de:02000:11943,Hamburg Elbbrücken,RV,"10,0245","53,5345",DB Station und Service AG,NaN
1,8001944,TETN,NaN,Eutingen Nord,RV,"8,7531","48,4847",DB Station und Service AG,NaN
2,8003074,MIA,NaN,Ingolstadt Audi,RV,"11,4074564","48,7904959",DB Station und Service AG,NaN
3,8001723,HEBA,NaN,Einbeck Otto-Hahn-Straße,RV,"9,89290953","51,8144784",Ilmebahn GmbH,NaN
4,8004371,KRO,NaN,Nörvenich-Rommelsheim,nur DPN,"6,547586","50,782539",Rurtalbahn GmbH,NaN
5,8010340,DSTR,NaN,Straßgräbchen-Bernsdorf,nur DPN,"14,052047","51,361469",NaN,NaN
6,8001510,TDSA,de:08237:8009:2,Dornstetten-Aach,RV,"8,48291","48,4733",DB Station und Service AG,NaN
7,8001966,MFOL,de:09187:90183,Feldolling,nur DPN,"11,852244","47,895336",DB Station und Service AG,NaN
8,8002060,FFGG,de:06412:11500,Frankfurt(Main)-Gateway Gardens,RV,"8,59449506","50,0565739",DB Station und Service AG,NaN
9,8002535,EOBG,de:05962:3517,Halver-Oberbrügge,RV,"7,574042","51,191867",DB Station und Service AG,NaN


In [6]:
df_tmp.dropna(subset=df.columns.to_list(), inplace=True)
df_tmp

/var/folders/jt/lp68flw167df0vb5tj2dgvfc0000gn/T/ipykernel_1672/3201845954.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp.dropna(subset=df.columns.to_list(), inplace=True)


,EVA_NR,DS100,IFOPT,NAME,Verkehr,Laenge,Breite,Betreiber_Name,Betreiber_Nr
24,8000001,KA,de:05334:1008,Aachen Hbf,FV,"6,091499","50,7678",DB Station und Service AG,1.0
25,8000002,TA,de:08136:1000,Aalen Hbf,FV,"10,096271","48,841013",DB Station und Service AG,4.0
26,8000004,HA,de:05774:7131,Altenbeken,FV,"8,943319","51,766433",DB Station und Service AG,85.0
27,8000007,FALZ,de:07331:55,Alzey,RV,"8,109749","49,7502",DB Station und Service AG,133.0
28,8000009,NAN,de:09561:11000,Ansbach,FV,"10,578239","49,298032",DB Station und Service AG,161.0
29,8000010,NAH,de:09661:99082,Aschaffenburg Hbf,FV,"9,143697","49,980557",DB Station und Service AG,187.0
30,8000011,AAG,de:01057:3365,Ascheberg(Holst),RV,"10,339481","54,148462",DB Station und Service AG,191.0
31,8000012,KAU,de:05382:59901,Au(Sieg),RV,"7,656537","50,773765",DB Station und Service AG,204.0
32,8000013,MA,de:09761:100,Augsburg Hbf,FV,"10,88557","48,365441",DB Station und Service AG,220.0
33,8000014,TAU,de:08436:1000,Aulendorf,FV,"9,644046","47,953052",DB Station und Service AG,226.0


In [7]:
def to_float(s: str) -> int:
    s = s.replace(',', '.')
    return float(s)
print(to_float('6,091499'))

6.091499


In [8]:
df_tmp['Laenge'] = df_tmp['Laenge'].apply(to_float)
df_tmp['Breite'] = df_tmp['Breite'].apply(to_float)
df_tmp = df_tmp[df_tmp['Laenge'].between(-90., 90.) & df_tmp['Breite'].between(-90., 90.)]
df_tmp

/var/folders/jt/lp68flw167df0vb5tj2dgvfc0000gn/T/ipykernel_1672/3030131991.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['Laenge'] = df_tmp['Laenge'].apply(to_float)
/var/folders/jt/lp68flw167df0vb5tj2dgvfc0000gn/T/ipykernel_1672/3030131991.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['Breite'] = df_tmp['Breite'].apply(to_float)


,EVA_NR,DS100,IFOPT,NAME,Verkehr,Laenge,Breite,Betreiber_Name,Betreiber_Nr
24,8000001,KA,de:05334:1008,Aachen Hbf,FV,6.091499,50.767800,DB Station und Service AG,1.0
25,8000002,TA,de:08136:1000,Aalen Hbf,FV,10.096271,48.841013,DB Station und Service AG,4.0
26,8000004,HA,de:05774:7131,Altenbeken,FV,8.943319,51.766433,DB Station und Service AG,85.0
27,8000007,FALZ,de:07331:55,Alzey,RV,8.109749,49.750200,DB Station und Service AG,133.0
28,8000009,NAN,de:09561:11000,Ansbach,FV,10.578239,49.298032,DB Station und Service AG,161.0
29,8000010,NAH,de:09661:99082,Aschaffenburg Hbf,FV,9.143697,49.980557,DB Station und Service AG,187.0
30,8000011,AAG,de:01057:3365,Ascheberg(Holst),RV,10.339481,54.148462,DB Station und Service AG,191.0
31,8000012,KAU,de:05382:59901,Au(Sieg),RV,7.656537,50.773765,DB Station und Service AG,204.0
32,8000013,MA,de:09761:100,Augsburg Hbf,FV,10.885570,48.365441,DB Station und Service AG,220.0
33,8000014,TAU,de:08436:1000,Aulendorf,FV,9.644046,47.953052,DB Station und Service AG,226.0


In [13]:
pattern = r'\b\w{2}:\d+:\d+(:\d+)?\b'
df_tmp = df_tmp[df_tmp['IFOPT'].apply(str).str.contains(pattern)]
df_tmp


/var/folders/jt/lp68flw167df0vb5tj2dgvfc0000gn/T/ipykernel_1672/2594425062.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_tmp = df_tmp[df_tmp['IFOPT'].apply(str).str.contains(pattern)]


,EVA_NR,DS100,IFOPT,NAME,Verkehr,Laenge,Breite,Betreiber_Name,Betreiber_Nr
24,8000001,KA,de:05334:1008,Aachen Hbf,FV,6.091499,50.767800,DB Station und Service AG,1.0
25,8000002,TA,de:08136:1000,Aalen Hbf,FV,10.096271,48.841013,DB Station und Service AG,4.0
26,8000004,HA,de:05774:7131,Altenbeken,FV,8.943319,51.766433,DB Station und Service AG,85.0
27,8000007,FALZ,de:07331:55,Alzey,RV,8.109749,49.750200,DB Station und Service AG,133.0
28,8000009,NAN,de:09561:11000,Ansbach,FV,10.578239,49.298032,DB Station und Service AG,161.0
29,8000010,NAH,de:09661:99082,Aschaffenburg Hbf,FV,9.143697,49.980557,DB Station und Service AG,187.0
30,8000011,AAG,de:01057:3365,Ascheberg(Holst),RV,10.339481,54.148462,DB Station und Service AG,191.0
31,8000012,KAU,de:05382:59901,Au(Sieg),RV,7.656537,50.773765,DB Station und Service AG,204.0
32,8000013,MA,de:09761:100,Augsburg Hbf,FV,10.885570,48.365441,DB Station und Service AG,220.0
33,8000014,TAU,de:08436:1000,Aulendorf,FV,9.644046,47.953052,DB Station und Service AG,226.0


In [1]:
import pandas as pd
import logging as log
import urllib.request
import zipfile

In [24]:
def extract_data(url):
    # extract zip file and save as csvfile
    zip_file_path = 'mowesta-dataset.zip'
    
    urllib.request.urlretrieve(url, zip_file_path)

    # Extract the CSV file from the ZIP
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # Assuming the CSV file is named 'data.csv'
        csv_file_name = 'data.csv'
        extracted_csv_path = zip_ref.extract(csv_file_name)
        
    return extracted_csv_path

In [38]:
# Read the extracted CSV file into a Pandas DataFrame
extracted_csv_path = extract_data("https://www.mowesta.com/data/measure/mowesta-dataset-20221107.zip")
print(extracted_csv_path)
df = pd.read_csv(extracted_csv_path, delimiter=';', decimal=',', on_bad_lines='skip')
#log.info(f"Extract data from : {url}.")
df.head()

/Users/arupmac/Desktop/2023-SAKI-um59ipuh-sarker/project/data/data.csv


Geraet  Hersteller  \
1 Fairphone FP4 7 12.7 48.57466 8.96484 18289 0 24.6 Ja    145038        25.2   
                  13.0 48.57466 9.31641 18289 0 27.6 Ja     76476        27.3   
                  13.7 48.57466 8.96484 18289 0 23.4 Nein  151277        23.5   
                  16.9 48.57466 8.96484 18289 0 24.7 Nein  131033        24.7   
                  18.8 48.57466 9.31641 18289 0 25.2 Nein  151246        25.1   

                                                          Model     Monat  \
1 Fairphone FP4 7 12.7 48.57466 8.96484 18289 0 24.6 Ja      Ja  219032.0   
                  13.0 48.57466 9.31641 18289 0 27.6 Ja      Ja  507590.0   
                  13.7 48.57466 8.96484 18289 0 23.4 Nein  Nein  222263.0   
                  16.9 48.57466 8.96484 18289 0 24.7 Nein  Nein  246299.0   
                  18.8 48.57466 9.31641 18289 0 25.2 Nein    Ja  222765.0   

                                                           Temperatur in °C (DWD)  \
1 Fairphone FP4 7 12.7 48.57466 8.96484 18289 0 24.6 Ja                      25.3   
                  13.0 48.57466 9.31641 18289 0 27.6 Ja                      27.2   
                  13.7 48.57466 8.96484 18289 0 23.4 Nein                    23.5   
                  16.9 48.57466 8.96484 18289 0 24.7 Nein                    24.7   
                  18.8 48.57466 9.31641 18289 0 25.2 Nein                    25.0   

                                                          Latitude (WGS84)  \
1 Fairphone FP4 7 12.7 48.57466 8.96484 18289 0 24.6 Ja                 Ja   
                  13.0 48.57466 9.31641 18289 0 27.6 Ja               Nein   
                  13.7 48.57466 8.96484 18289 0 23.4 Nein             Nein   
                  16.9 48.57466 8.96484 18289 0 24.7 Nein             Nein   
                  18.8 48.57466 9.31641 18289 0 25.2 Nein               Ja   

                                                           Longitude (WGS84)  \
1 Fairphone FP4 7 12.7 48.57466 8.96484 18289 0 24.6 Ja             300020.0   
                  13.0 48.57466 9.31641 18289 0 27.6 Ja             620631.0   
                  13.7 48.57466 8.96484 18289 0 23.4 Nein           288249.0   
                  16.9 48.57466 8.96484 18289 0 24.7 Nein           312283.0   
                  18.8 48.57466 9.31641 18289 0 25.2 Nein           287631.0   

                                                           Verschleierung (m)  \
1 Fairphone FP4 7 12.7 48.57466 8.96484 18289 0 24.6 Ja                  25.4   
                  13.0 48.57466 9.31641 18289 0 27.6 Ja                  26.7   
                  13.7 48.57466 8.96484 18289 0 23.4 Nein                23.4   
                  16.9 48.57466 8.96484 18289 0 24.7 Nein                24.7   
                  18.8 48.57466 9.31641 18289 0 25.2 Nein                25.2   

                                                          Aufenthaltsdauer im Freien (ms)  \
1 Fairphone FP4 7 12.7 48.57466 8.96484 18289 0 24.6 Ja                                Ja   
                  13.0 48.57466 9.31641 18289 0 27.6 Ja                              Nein   
                  13.7 48.57466 8.96484 18289 0 23.4 Nein                            Nein   
                  16.9 48.57466 8.96484 18289 0 24.7 Nein                            Nein   
                  18.8 48.57466 9.31641 18289 0 25.2 Nein                            Nein   

                                                           Batterietemperatur in °C  \
1 Fairphone FP4 7 12.7 48.57466 8.96484 18289 0 24.6 Ja                    396370.0   
                  13.0 48.57466 9.31641 18289 0 27.6 Ja                         NaN   
                  13.7 48.57466 8.96484 18289 0 23.4 Nein                  356604.0   
                  16.9 48.57466 8.96484 18289 0 24.7 Nein                  377301.0   
                  18.8 48.57466 9.31641 18289 0 25.2 Nein                  348211.0   

                                                           Geraet aktiv  \
1 Fa

In [33]:
df.columns

Index(['Geraet', 'Hersteller', 'Model', 'Monat', 'Temperatur in °C (DWD)',
       'Latitude (WGS84)', 'Longitude (WGS84)', 'Verschleierung (m)',
       'Aufenthaltsdauer im Freien (ms)', 'Batterietemperatur in °C',
       'Geraet aktiv', 'Aufenthaltsdauer im Freien (ms).1',
       'Batterietemperatur in °C.1', 'Geraet aktiv.1', '...', 'Unnamed: 15'],
      dtype='object')

In [34]:
selected_columns = [
    'Geraet',
    'Hersteller',
    'Model',
    'Monat',
    'Temperatur in °C (DWD)',
    'Batterietemperatur in °C',
    'Geraet aktiv'
]
df = df[selected_columns]
df.columns

Index(['Geraet', 'Hersteller', 'Model', 'Monat', 'Temperatur in °C (DWD)',
       'Batterietemperatur in °C', 'Geraet aktiv'],
      dtype='object')

In [37]:
df['Geraet']

1   Fairphone  FP4       7   12.7  48.57466  8.96484  18289  0  24.6  Ja      145038
                             13.0  48.57466  9.31641  18289  0  27.6  Ja       76476
                             13.7  48.57466  8.96484  18289  0  23.4  Nein    151277
                             16.9  48.57466  8.96484  18289  0  24.7  Nein    131033
                             18.8  48.57466  9.31641  18289  0  25.2  Nein    151246
                                                                               ...  
93  samsung    SM-G960F  2   2.6   51.28927  7.20703  17287  0  27.7  Ja      182123
97  samsung    SM-G960F  1  -1.5   50.17677  7.20703  17703  0  29.4  Ja      115197
98  samsung    SM-G960F  1  -1.3   50.17677  7.20703  17703  0  30.7  Ja      303381
                             4.7   50.17677  7.20703  17703  0  28.5  Nein    121277
                                                                28.9  Nein    406674
Name: Geraet, Length: 2224, dtype: int64

In [23]:
import pandas as pd
import logging as log
import urllib.request
import zipfile
import os

zip_file_path = 'mowesta-dataset.zip'
    
urllib.request.urlretrieve("https://www.mowesta.com/data/measure/mowesta-dataset-20221107.zip", zip_file_path)

# Extract the CSV file from the ZIP
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Assuming the CSV file is named 'data.csv'
    csv_file_name = 'data.csv'
    extracted_csv_path = zip_ref.extract(csv_file_name)

extracted_csv_path = "/Users/arupmac/Desktop/2023-SAKI-um59ipuh-sarker/project/data/data.csv"
selected_columns = [
        'Geraet',
        'Hersteller',
        'Model',
        'Monat',
        'Temperatur in °C (DWD)',
        'Batterietemperatur in °C',
        'Geraet aktiv'
    ]
    
intended_types = {
    'Temperatur in °C (DWD)': 'float',
    'Batterietemperatur in °C': 'float'
}
names_ = ["Geraet","Hersteller","Model","Monat","Temperatur in °C (DWD)","Latitude (WGS84)","Longitude (WGS84)",
         "Verschleierung (m)","Aufenthaltsdauer im Freien (ms)","Batterietemperatur in °C","Geraet aktiv","Aufenthaltsdauer im Freien (ms)",
         "Batterietemperatur in °C","Geraet aktiv","..."]
# Read the extracted CSV file into a Pandas DataFrame
df = pd.read_csv(extracted_csv_path, delimiter=";", decimal=',', 
                dtype=intended_types, header=0, names=names_, usecols=selected_columns)
df.head()

ValueError: Duplicate names are not allowed.

In [ ]:
# Rename columns
column_mapping = {
    'Temperatur in °C (DWD)': 'Temperatur',
    'Batterietemperatur in °C': 'Batterietemperatur'
}
df = df.rename(columns=column_mapping)

In [89]:
df = df.loc[:,~df.columns.duplicated()].copy()
df.head()

,Geraet,Hersteller,Model,Monat,Temperatur,Batterietemperatur,Geraet aktiv
1,2,samsung,SM-A515F,6,"21,6","25,8",Nein
2,3,samsung,SM-J510FN,6,"22,5","29,9",Nein
3,4,samsung,SM-G991B,5,"18,6","29,6",Ja
4,8,Google,Pixel 4a,3,"16,7","28,3",Ja
5,8,Google,Pixel 4a,4,"12,8","30,3",Nein


In [90]:
df['Temperatur'] = (df['Temperatur'].str.replace(',','.').apply(float) * 9/5) + 32
df['Batterietemperatur'] = (df['Batterietemperatur'].str.replace(',','.').apply(float) * 9/5) + 32
df.head()

,Geraet,Hersteller,Model,Monat,Temperatur,Batterietemperatur,Geraet aktiv
1,2,samsung,SM-A515F,6,70.88,78.44,Nein
2,3,samsung,SM-J510FN,6,72.50,85.82,Nein
3,4,samsung,SM-G991B,5,65.48,85.28,Ja
4,8,Google,Pixel 4a,3,62.06,82.94,Ja
5,8,Google,Pixel 4a,4,55.04,86.54,Nein


In [92]:
# Validate data
df = df[df['Geraet'].apply(int) > 0]
df.head()

,Geraet,Hersteller,Model,Monat,Temperatur,Batterietemperatur,Geraet aktiv
1,2,samsung,SM-A515F,6,70.88,78.44,Nein
2,3,samsung,SM-J510FN,6,72.50,85.82,Nein
3,4,samsung,SM-G991B,5,65.48,85.28,Ja
4,8,Google,Pixel 4a,3,62.06,82.94,Ja
5,8,Google,Pixel 4a,4,55.04,86.54,Nein


In [93]:
import sqlite3

# Define SQLite types for each column
sqlite_types = {
    'Geraet': 'BIGINT',
    'Hersteller': 'TEXT',
    'Model': 'TEXT',
    'Monat': 'TEXT',
    'Temperatur': 'FLOAT',
    'Batterietemperatur': 'FLOAT',
    'Geraet aktiv': 'BIGINT'
}

# Write data to SQLite database
database_path = 'temperatures.sqlite'
table_name = 'temperatures'

with sqlite3.connect(database_path) as conn:
    df.to_sql(table_name, conn, index=False, if_exists='replace', dtype=sqlite_types)

In [ ]:
import csv
extracted_csv_path = "/Users/arupmac/Desktop/2023-SAKI-um59ipuh-sarker/project/data/data.csv"
with open(extracted_csv_path, 'r') as file:
    # Create a CSV reader object
    csv_reader = csv.reader(file)

    print(len(csv_reader))